In [112]:
import quimb.tensor as qtn
import quimb as qu
import numpy as np
import math
import random

In [113]:
N = 32
s = ""
for i in range(0, N):
    r = random.randint(0, 1)
    if r == 0:
        s = s + "0"
    else:
        s = s + "1"
print(s)
compress = True
def check_bit_in_s(s, i, k):
    return (s[k] == i)

10110100111001101000110100111001


In [114]:
# Mean operator

t0 = np.zeros((2,2,2))
t0[0][0][0] = 1
t0[1][1][1] = 1

t_datas = [t0]

for i in range(1, N-1):
    t_k = np.zeros((2,2,2,2))
    t_k[0][0][0][0] = 1
    t_k[0][1][1][1] = 1
    t_k[1][0][0][1] = 1
    t_k[1][1][1][1] = 1
    t_datas.append(t_k)

tn = np.zeros((2,2,2))
tn[0][0][0] = 1
tn[0][1][1] = -1
tn[1][0][0] = -1
tn[1][1][1] = -1

t_datas.append(tn)

tensors = []

tensor_0 = qtn.Tensor(data=t0, inds=["i_0", "j_0","0"], tags=("I_0"))
tensors.append(tensor_0)

for i in range(1, N-1):
    tensors.append(qtn.Tensor(data=t_datas[i], inds=["{}".format(i-1), "i_{}".format(i), "j_{}".format(i), "{}".format(i)], tags=("I_{}".format(i))))

tensor_n = qtn.Tensor(data=tn, inds=["{}".format(N-2), "i_{}".format(N-1), "j_{}".format(N-1)], tags=("I_{}".format(N-1)))
tensors.append(tensor_n)
tn = qtn.TensorNetwork(tensors)
# tn.draw(show_inds='all', show_tags = True)

In [115]:
mean_mpo = qtn.MatrixProductOperator.from_TN(tn,
site_tag_id='I_{}',
    upper_ind_id='j_{}',
    lower_ind_id='i_{}',
    cyclic=False,
    L=N)

In [116]:
hadamard_matrix = np.array([[1,1],[1,-1]]) * 1/np.sqrt(2)
hadamard_tensors = []
for m in range(0, N):
    hadamard_tensors.append(qtn.Tensor(data=hadamard_matrix, inds=["hi_{}".format(m), "hj_{}".format(m)], tags=("I_{}".format(m))))

hadamard_tn = qtn.TensorNetwork(hadamard_tensors)

hadamard_mpo = qtn.MatrixProductOperator.from_TN(hadamard_tn,
site_tag_id='I_{}',
    upper_ind_id='hj_{}',
    lower_ind_id='hi_{}',
    cyclic=False,
    L=N)

In [117]:
diffusion_mpo = mean_mpo.apply(hadamard_mpo, compress=compress)
diffusion_mpo = hadamard_mpo.apply(diffusion_mpo, compress=compress)

In [118]:
# Oracle

to0 = np.zeros((2,2,2))
to0[0][0][0] = check_bit_in_s(s, '0', 0)
to0[1][1][0] = check_bit_in_s(s, '1', 0)
to0[1][1][1] = (1 - check_bit_in_s(s, '1', 0))
to0[0][0][1] = (1 - check_bit_in_s(s, '0', 0))

to_datas = [to0]

for i in range(1, N-1):
    t_k = np.zeros((2,2,2,2))
    t_k[0][0][0][0] = check_bit_in_s(s, '0', i)
    t_k[0][0][0][1] = (1 - check_bit_in_s(s, '0', i))
    t_k[0][1][1][0] = check_bit_in_s(s, '1', i)
    t_k[0][1][1][1] = (1 - check_bit_in_s(s, '1', i))
    t_k[1][0][0][1] = 1
    t_k[1][1][1][1] = 1
    to_datas.append(t_k)

ton = np.zeros((2,2,2))
ton[0][0][0] = (-1 * check_bit_in_s(s, '0', N-1) + (1 - check_bit_in_s(s, '0', N-1)))
ton[0][1][1] = (-1 * check_bit_in_s(s, '1', N-1) + (1 - check_bit_in_s(s, '1', N-1)))
ton[1][0][0] = 1
ton[1][1][1] = 1

to_datas.append(ton)

oracle_tensors = []

oracle_tensor_0 = qtn.Tensor(data=to0, inds=["oi_0", "oj_0","0"], tags=("I_0"))
oracle_tensors.append(oracle_tensor_0)

for i in range(1, N-1):
    oracle_tensors.append(qtn.Tensor(data=to_datas[i], inds=["{}".format(i-1), "oi_{}".format(i), "oj_{}".format(i), "{}".format(i)], tags=("I_{}".format(i))))

oracle_tensor_n = qtn.Tensor(data=ton, inds=["{}".format(N-2), "oi_{}".format(N-1), "oj_{}".format(N-1)], tags=("I_{}".format(N-1)))
oracle_tensors.append(oracle_tensor_n)
oracle_tn = qtn.TensorNetwork(oracle_tensors)
# oracle_tn.draw(show_inds='all', show_tags = True)

In [119]:
oracle_mpo = qtn.MatrixProductOperator.from_TN(oracle_tn,
site_tag_id='I_{}',
    upper_ind_id='oj_{}',
    lower_ind_id='oi_{}',
    cyclic=False,
    L=N)

In [120]:
operator_mpo = diffusion_mpo.apply(oracle_mpo, compress=compress)

In [121]:
iters = math.floor((math.pi/4 * math.sqrt((2 ** N))))

In [122]:
# Init State after first hadamard

init_t0 = np.array([1, 1]) * 1/np.sqrt(2)

init_tensors = []

for i in range(0, N):
    init_tensors.append(qtn.Tensor(data=init_t0, inds=["ii_{}".format(i)], tags=("I_{}".format(i))))

init_tn = qtn.TensorNetwork(init_tensors)
# init_tn.draw(show_inds='all', show_tags = True)

init_mps = qtn.MatrixProductState.from_TN(init_tn,
site_ind_id='ii_{}',
site_tag_id='I_{}',
    cyclic=False,
    L=N)

In [123]:
def multiply_operator(hash_map, iters, oper):
    # print(iters, iters in hash_map)
    if iters in hash_map:
        return hash_map[iters]
    if iters == 1:
        hash_map[iters] = oper
        return oper
    if iters == 2:
        op = oper.apply(oper, compress=compress)
        hash_map[iters] = op
        return op
    op1 = multiply_operator(hash_map, iters//2, oper)
    op2 = multiply_operator(hash_map, iters - iters//2, oper)

    op = op1.apply(op2, compress=compress)
    hash_map[iters] = op
    return op

In [124]:
hash_map = {}
# iters = 10
# final_operator_mpo = multiply_operator(hash_map, iters, operator_mpo)
# final_mps = final_operator_mpo.apply(init_mps, compress=compress)
# final_mps = final_mps * 1/np.sqrt(2)


In [125]:
# a = final_mps.to_dense()
# print(s)
# print(a[int(s, 2)])
# print(a[0])

In [126]:
final_mps = init_mps
for i in range(0, iters):
    final_mps = operator_mpo.apply(final_mps, compress=compress)

In [127]:
# b = ""
# is_correct = False
# for j in range(0, 10):
#     out = final_mps
#     for i in range(0, N):
#         (sampled_s, out) = out.measure(i)
#         b += str(sampled_s)

#     if b == s:
#         is_correct = True
#         break

# print(is_correct, s, b)

In [128]:
circ = qtn.Circuit(psi0=final_mps)

In [129]:
sampled_s = ""
is_correct = False
print(s)
for b in circ.sample(100):
    if b == s:
        sampled_s = b
        is_correct = True
        break
print(is_correct, s, sampled_s, len(s))

10110100111001101000110100111001
True 10110100111001101000110100111001 10110100111001101000110100111001 32


In [130]:


# is_correct = False
# sampled_s = ""
# for i in range(0,10):
#     index = np.random.choice(len(result), p=result)
#     index_s = bin(index)[2:].zfill(N)
#     # print(index_s)
#     if index_s == s:
#         is_correct = True 
#         sampled_s = index_s
#         break

# print(is_correct, s, sampled_s)